In [1]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetections2 import DecodeDetections2
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

C:\Users\Alok Singh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_height = 300 # Height of the input images
img_width = 300 # Width of the input images
img_channels = 3 # Number of color channels of the input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR.

n_classes = 4 # Number of positive classes, for eg: 1 if training on a human(pedestrian) dataset or n_classes=n where n is the number of different people in the photos.

scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip ground truth and anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

In [3]:
# 1: Build the Keras model.

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)

# 2: Load some weights into the model.

# Set the path to the weights you want to load.
weights_path = 'C:/Users/Alok Singh/Desktop/Face detection and recognition/Face recognition trained weights.h5'

model.load_weights(weights_path, by_name=True)

# 3: Instantiate an Adam optimizer and the SSD loss function and compile the model.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=5e-04)

ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [5]:
# 1: Instantiate to `BatchGenerator` objects: One for training, one for validation.

train_dataset = DataGenerator()
val_dataset = DataGenerator()

# 2: Parse the image and label lists for the training and validation datasets. This can take a while.

# Set the paths to the datasets here.

# The directories that contain the images, these folders are created while creating the dataset.
VOC_2007_images_dir      = 'D:/face detection and recognition/alok/FaceVOC/JPEGImages/'

# The directories that contain the annotations.
VOC_2007_annotations_dir      = 'D:/face detection and recognition/alok/FaceVOC/Annotations/'

# The paths to the image sets.
VOC_2007_train_image_set_filename    = 'D:/face detection and recognition/alok/FaceVOC/Main/train.txt'
VOC_2007_val_image_set_filename      = 'D:/face detection and recognition/alok/FaceVOC/Main/val.txt'
VOC_2007_trainval_image_set_filename = 'D:/face detection and recognition/alok/FaceVOC/Main/trainval.txt'
VOC_2007_test_image_set_filename     = 'D:/face detection and recognition/alok/FaceVOC/Main/test.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
# Set the name of the classes, in case of pedestrian uncomment the 'pedestrian' class name below and comment the upper classes name

    
classes = ['background','alok','harsh','rahul','varadaraja']
#classes = ['background','pedestrian']


train_dataset.parse_xml(images_dirs=[VOC_2007_images_dir
                                    ],
                        image_set_filenames=[VOC_2007_trainval_image_set_filename],
                        annotations_dirs=[VOC_2007_annotations_dir],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False)

val_dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                      image_set_filenames=[VOC_2007_test_image_set_filename],
                      annotations_dirs=[VOC_2007_annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=False,
                      ret=False)

test.txt: 100%|███████████████████████████████████████████████████████████████████| 1254/1254 [00:05<00:00, 247.23it/s]


In [6]:
# 3: Set the batch size.

batch_size = 1 # Change the batch size if you run into memory issues with GPU.

# 4: Set the image transformations for pre-processing and data augmentation options.

# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.4,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	  1234
Number of images in the validation dataset:	  1254


In [7]:
# Define a learning rate schedule.

def lr_schedule(epoch):
    if epoch < 80:
        return 0.001
    elif epoch < 100:
        return 0.0001
    else:
        return 0.00001

In [8]:
# Define model callbacks.

# Set the filepath under which you want to save the weights.
model_checkpoint = ModelCheckpoint(filepath='ssd300_weights_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=True,
                                   mode='auto',
                                   period=1)

learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule,
                                                verbose=1)

callbacks = [model_checkpoint,
             learning_rate_scheduler]

In [9]:
epochs = 200
steps_per_epoch = 500

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size))

Epoch 1/200

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
321/500 [==================>...........] - ETA: 53:24 - loss: 1.63 - ETA: 27:27 - loss: 5.05 - ETA: 18:48 - loss: 5.82 - ETA: 14:28 - loss: 8.11 - ETA: 11:53 - loss: 9.83 - ETA: 10:09 - loss: 10.401 - ETA: 8:54 - loss: 10.382 - ETA: 7:59 - loss: 9.7577 - ETA: 7:15 - loss: 9.293 - ETA: 6:40 - loss: 9.506 - ETA: 6:12 - loss: 9.955 - ETA: 5:48 - loss: 9.780 - ETA: 5:28 - loss: 9.609 - ETA: 5:10 - loss: 9.729 - ETA: 4:55 - loss: 9.091 - ETA: 4:42 - loss: 8.927 - ETA: 4:31 - loss: 8.715 - ETA: 4:20 - loss: 8.950 - ETA: 4:11 - loss: 8.487 - ETA: 4:02 - loss: 8.387 - ETA: 3:55 - loss: 8.286 - ETA: 3:48 - loss: 8.112 - ETA: 3:42 - loss: 8.001 - ETA: 3:36 - loss: 7.963 - ETA: 3:30 - loss: 8.032 - ETA: 3:25 - loss: 8.020 - ETA: 3:21 - loss: 7.960 - ETA: 3:16 - loss: 7.980 - ETA: 3:12 - loss: 7.885 - ETA: 3:09 - loss: 7.775 - ETA: 3:05 - loss: 7.799 - ETA: 3:02 - loss: 7.765 - ETA: 2:59 - loss: 7.793 - ETA: 2:56 - lo

324/500 [==================>...........] - ETA: 1:38 - loss: 5.466 - ETA: 1:37 - loss: 5.825 - ETA: 1:37 - loss: 6.147 - ETA: 1:37 - loss: 6.808 - ETA: 1:37 - loss: 6.608 - ETA: 1:37 - loss: 6.720 - ETA: 1:36 - loss: 6.372 - ETA: 1:36 - loss: 6.244 - ETA: 1:36 - loss: 5.975 - ETA: 1:36 - loss: 6.026 - ETA: 1:36 - loss: 6.041 - ETA: 1:36 - loss: 6.034 - ETA: 1:36 - loss: 6.249 - ETA: 1:36 - loss: 6.151 - ETA: 1:36 - loss: 6.160 - ETA: 1:35 - loss: 6.066 - ETA: 1:35 - loss: 5.984 - ETA: 1:35 - loss: 5.872 - ETA: 1:35 - loss: 5.989 - ETA: 1:34 - loss: 6.072 - ETA: 1:34 - loss: 6.005 - ETA: 1:34 - loss: 5.990 - ETA: 1:34 - loss: 5.922 - ETA: 1:33 - loss: 5.874 - ETA: 1:33 - loss: 5.937 - ETA: 1:33 - loss: 5.931 - ETA: 1:33 - loss: 5.913 - ETA: 1:33 - loss: 5.880 - ETA: 1:32 - loss: 5.841 - ETA: 1:32 - loss: 5.917 - ETA: 1:32 - loss: 5.735 - ETA: 1:32 - loss: 5.803 - ETA: 1:32 - loss: 5.847 - ETA: 1:31 - loss: 5.882 - ETA: 1:31 - loss: 5.883 - ETA: 1:31 - loss: 5.863 - ETA: 1:31 - loss: 5.8

500/500 [==============================] - ETA: 34s - loss: 6.36 - ETA: 34s - loss: 6.36 - ETA: 34s - loss: 6.36 - ETA: 33s - loss: 6.36 - ETA: 33s - loss: 6.36 - ETA: 33s - loss: 6.35 - ETA: 33s - loss: 6.35 - ETA: 33s - loss: 6.35 - ETA: 32s - loss: 6.35 - ETA: 32s - loss: 6.36 - ETA: 32s - loss: 6.36 - ETA: 32s - loss: 6.35 - ETA: 32s - loss: 6.36 - ETA: 31s - loss: 6.35 - ETA: 31s - loss: 6.36 - ETA: 31s - loss: 6.35 - ETA: 31s - loss: 6.35 - ETA: 31s - loss: 6.36 - ETA: 30s - loss: 6.35 - ETA: 30s - loss: 6.36 - ETA: 30s - loss: 6.36 - ETA: 30s - loss: 6.36 - ETA: 30s - loss: 6.36 - ETA: 29s - loss: 6.36 - ETA: 29s - loss: 6.36 - ETA: 29s - loss: 6.36 - ETA: 29s - loss: 6.36 - ETA: 29s - loss: 6.35 - ETA: 28s - loss: 6.35 - ETA: 28s - loss: 6.36 - ETA: 28s - loss: 6.35 - ETA: 28s - loss: 6.35 - ETA: 28s - loss: 6.35 - ETA: 27s - loss: 6.35 - ETA: 27s - loss: 6.35 - ETA: 27s - loss: 6.35 - ETA: 27s - loss: 6.35 - ETA: 27s - loss: 6.35 - ETA: 26s - loss: 6.35 - ETA: 26s - loss: 6.35

324/500 [==================>...........] - ETA: 1:37 - loss: 8.209 - ETA: 1:37 - loss: 7.909 - ETA: 1:37 - loss: 6.527 - ETA: 1:37 - loss: 6.256 - ETA: 1:37 - loss: 5.707 - ETA: 1:37 - loss: 5.548 - ETA: 1:37 - loss: 5.356 - ETA: 1:37 - loss: 5.614 - ETA: 1:36 - loss: 5.919 - ETA: 1:36 - loss: 5.353 - ETA: 1:36 - loss: 5.440 - ETA: 1:36 - loss: 5.650 - ETA: 1:35 - loss: 5.724 - ETA: 1:35 - loss: 5.767 - ETA: 1:35 - loss: 5.400 - ETA: 1:35 - loss: 6.057 - ETA: 1:35 - loss: 6.041 - ETA: 1:34 - loss: 5.988 - ETA: 1:34 - loss: 5.987 - ETA: 1:34 - loss: 5.989 - ETA: 1:34 - loss: 5.956 - ETA: 1:33 - loss: 5.836 - ETA: 1:33 - loss: 5.853 - ETA: 1:33 - loss: 5.858 - ETA: 1:33 - loss: 5.863 - ETA: 1:33 - loss: 5.779 - ETA: 1:32 - loss: 5.744 - ETA: 1:32 - loss: 5.548 - ETA: 1:32 - loss: 5.466 - ETA: 1:32 - loss: 5.515 - ETA: 1:32 - loss: 5.449 - ETA: 1:31 - loss: 5.435 - ETA: 1:31 - loss: 5.452 - ETA: 1:31 - loss: 5.448 - ETA: 1:31 - loss: 5.449 - ETA: 1:31 - loss: 5.482 - ETA: 1:30 - loss: 5.4

500/500 [==============================] - ETA: 34s - loss: 5.02 - ETA: 34s - loss: 5.02 - ETA: 34s - loss: 5.00 - ETA: 33s - loss: 5.00 - ETA: 33s - loss: 5.00 - ETA: 33s - loss: 5.01 - ETA: 33s - loss: 5.01 - ETA: 33s - loss: 5.01 - ETA: 32s - loss: 5.00 - ETA: 32s - loss: 5.00 - ETA: 32s - loss: 5.01 - ETA: 32s - loss: 5.02 - ETA: 32s - loss: 5.01 - ETA: 31s - loss: 4.99 - ETA: 31s - loss: 5.01 - ETA: 31s - loss: 5.02 - ETA: 31s - loss: 5.02 - ETA: 31s - loss: 5.03 - ETA: 30s - loss: 5.03 - ETA: 30s - loss: 5.03 - ETA: 30s - loss: 5.02 - ETA: 30s - loss: 5.02 - ETA: 30s - loss: 5.02 - ETA: 29s - loss: 5.02 - ETA: 29s - loss: 5.02 - ETA: 29s - loss: 5.02 - ETA: 29s - loss: 5.01 - ETA: 29s - loss: 5.01 - ETA: 28s - loss: 5.01 - ETA: 28s - loss: 5.01 - ETA: 28s - loss: 5.01 - ETA: 28s - loss: 5.02 - ETA: 28s - loss: 5.02 - ETA: 27s - loss: 5.02 - ETA: 27s - loss: 5.01 - ETA: 27s - loss: 5.01 - ETA: 27s - loss: 5.01 - ETA: 27s - loss: 5.01 - ETA: 26s - loss: 5.01 - ETA: 26s - loss: 5.00

...

500/500 [==============================] - ETA: 34s - loss: 1.54 - ETA: 33s - loss: 1.54 - ETA: 33s - loss: 1.54 - ETA: 33s - loss: 1.53 - ETA: 33s - loss: 1.56 - ETA: 33s - loss: 1.58 - ETA: 32s - loss: 1.57 - ETA: 32s - loss: 1.57 - ETA: 32s - loss: 1.57 - ETA: 32s - loss: 1.57 - ETA: 32s - loss: 1.58 - ETA: 31s - loss: 1.57 - ETA: 31s - loss: 1.57 - ETA: 31s - loss: 1.59 - ETA: 31s - loss: 1.59 - ETA: 31s - loss: 1.58 - ETA: 31s - loss: 1.58 - ETA: 30s - loss: 1.58 - ETA: 30s - loss: 1.58 - ETA: 30s - loss: 1.57 - ETA: 30s - loss: 1.57 - ETA: 30s - loss: 1.57 - ETA: 29s - loss: 1.57 - ETA: 29s - loss: 1.56 - ETA: 29s - loss: 1.57 - ETA: 29s - loss: 1.56 - ETA: 29s - loss: 1.56 - ETA: 28s - loss: 1.56 - ETA: 28s - loss: 1.56 - ETA: 28s - loss: 1.56 - ETA: 28s - loss: 1.56 - ETA: 28s - loss: 1.58 - ETA: 27s - loss: 1.57 - ETA: 27s - loss: 1.57 - ETA: 27s - loss: 1.57 - ETA: 27s - loss: 1.57 - ETA: 27s - loss: 1.57 - ETA: 26s - loss: 1.57 - ETA: 26s - loss: 1.59 - ETA: 26s - loss: 1.59

324/500 [==================>...........] - ETA: 1:37 - loss: 0.698 - ETA: 1:38 - loss: 0.731 - ETA: 1:37 - loss: 1.660 - ETA: 1:37 - loss: 1.397 - ETA: 1:37 - loss: 1.698 - ETA: 1:37 - loss: 1.494 - ETA: 1:36 - loss: 1.376 - ETA: 1:36 - loss: 1.260 - ETA: 1:36 - loss: 1.405 - ETA: 1:36 - loss: 1.310 - ETA: 1:36 - loss: 1.238 - ETA: 1:35 - loss: 1.222 - ETA: 1:35 - loss: 1.210 - ETA: 1:35 - loss: 1.206 - ETA: 1:35 - loss: 1.141 - ETA: 1:35 - loss: 1.174 - ETA: 1:35 - loss: 1.128 - ETA: 1:34 - loss: 1.093 - ETA: 1:34 - loss: 1.073 - ETA: 1:34 - loss: 1.040 - ETA: 1:34 - loss: 1.024 - ETA: 1:34 - loss: 1.021 - ETA: 1:33 - loss: 2.394 - ETA: 1:33 - loss: 2.313 - ETA: 1:33 - loss: 2.256 - ETA: 1:33 - loss: 2.292 - ETA: 1:33 - loss: 2.220 - ETA: 1:32 - loss: 2.158 - ETA: 1:32 - loss: 2.192 - ETA: 1:32 - loss: 2.153 - ETA: 1:32 - loss: 2.108 - ETA: 1:32 - loss: 2.057 - ETA: 1:32 - loss: 2.088 - ETA: 1:31 - loss: 2.033 - ETA: 1:31 - loss: 2.211 - ETA: 1:31 - loss: 2.174 - ETA: 1:31 - loss: 2.1

500/500 [==============================] - ETA: 34s - loss: 1.54 - ETA: 34s - loss: 1.53 - ETA: 34s - loss: 1.53 - ETA: 34s - loss: 1.53 - ETA: 33s - loss: 1.54 - ETA: 33s - loss: 1.53 - ETA: 33s - loss: 1.54 - ETA: 33s - loss: 1.54 - ETA: 33s - loss: 1.54 - ETA: 32s - loss: 1.54 - ETA: 32s - loss: 1.53 - ETA: 32s - loss: 1.53 - ETA: 32s - loss: 1.53 - ETA: 32s - loss: 1.53 - ETA: 31s - loss: 1.52 - ETA: 31s - loss: 1.53 - ETA: 31s - loss: 1.53 - ETA: 31s - loss: 1.52 - ETA: 31s - loss: 1.52 - ETA: 30s - loss: 1.52 - ETA: 30s - loss: 1.52 - ETA: 30s - loss: 1.52 - ETA: 30s - loss: 1.51 - ETA: 30s - loss: 1.51 - ETA: 29s - loss: 1.51 - ETA: 29s - loss: 1.52 - ETA: 29s - loss: 1.52 - ETA: 29s - loss: 1.51 - ETA: 29s - loss: 1.53 - ETA: 28s - loss: 1.53 - ETA: 28s - loss: 1.53 - ETA: 28s - loss: 1.53 - ETA: 28s - loss: 1.52 - ETA: 28s - loss: 1.52 - ETA: 27s - loss: 1.52 - ETA: 27s - loss: 1.52 - ETA: 27s - loss: 1.51 - ETA: 27s - loss: 1.51 - ETA: 27s - loss: 1.51 - ETA: 26s - loss: 1.51

324/500 [==================>...........] - ETA: 1:37 - loss: 0.497 - ETA: 1:36 - loss: 0.450 - ETA: 1:37 - loss: 0.752 - ETA: 1:37 - loss: 0.682 - ETA: 1:37 - loss: 0.640 - ETA: 1:36 - loss: 0.937 - ETA: 1:36 - loss: 0.877 - ETA: 1:36 - loss: 1.091 - ETA: 1:36 - loss: 1.068 - ETA: 1:35 - loss: 1.009 - ETA: 1:35 - loss: 1.761 - ETA: 1:35 - loss: 1.669 - ETA: 1:35 - loss: 1.630 - ETA: 1:34 - loss: 1.581 - ETA: 1:34 - loss: 1.551 - ETA: 1:34 - loss: 1.491 - ETA: 1:34 - loss: 1.472 - ETA: 1:34 - loss: 1.421 - ETA: 1:34 - loss: 1.378 - ETA: 1:33 - loss: 1.334 - ETA: 1:33 - loss: 1.304 - ETA: 1:33 - loss: 1.271 - ETA: 1:33 - loss: 1.289 - ETA: 1:33 - loss: 1.436 - ETA: 1:32 - loss: 1.397 - ETA: 1:32 - loss: 1.466 - ETA: 1:32 - loss: 1.440 - ETA: 1:32 - loss: 1.426 - ETA: 1:32 - loss: 1.392 - ETA: 1:31 - loss: 1.371 - ETA: 1:31 - loss: 1.349 - ETA: 1:31 - loss: 1.328 - ETA: 1:31 - loss: 1.383 - ETA: 1:31 - loss: 1.359 - ETA: 1:30 - loss: 1.348 - ETA: 1:30 - loss: 1.332 - ETA: 1:30 - loss: 1.3

500/500 [==============================] - ETA: 34s - loss: 1.49 - ETA: 34s - loss: 1.49 - ETA: 33s - loss: 1.49 - ETA: 33s - loss: 1.50 - ETA: 33s - loss: 1.51 - ETA: 33s - loss: 1.51 - ETA: 33s - loss: 1.51 - ETA: 32s - loss: 1.51 - ETA: 32s - loss: 1.51 - ETA: 32s - loss: 1.50 - ETA: 32s - loss: 1.50 - ETA: 32s - loss: 1.50 - ETA: 31s - loss: 1.50 - ETA: 31s - loss: 1.50 - ETA: 31s - loss: 1.52 - ETA: 31s - loss: 1.52 - ETA: 31s - loss: 1.52 - ETA: 30s - loss: 1.53 - ETA: 30s - loss: 1.52 - ETA: 30s - loss: 1.52 - ETA: 30s - loss: 1.52 - ETA: 30s - loss: 1.52 - ETA: 29s - loss: 1.51 - ETA: 29s - loss: 1.51 - ETA: 29s - loss: 1.51 - ETA: 29s - loss: 1.51 - ETA: 29s - loss: 1.51 - ETA: 28s - loss: 1.51 - ETA: 28s - loss: 1.51 - ETA: 28s - loss: 1.51 - ETA: 28s - loss: 1.50 - ETA: 28s - loss: 1.50 - ETA: 27s - loss: 1.50 - ETA: 27s - loss: 1.52 - ETA: 27s - loss: 1.52 - ETA: 27s - loss: 1.51 - ETA: 27s - loss: 1.52 - ETA: 26s - loss: 1.53 - ETA: 26s - loss: 1.53 - ETA: 26s - loss: 1.53